In [ ]:
# %pylab nbagg
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

from lab_repo.classes.dbclasses import dbExperimentSet, dbExperiment
from lab_repo.classes import pcExperimentGroup

import numpy as np
import scipy
from scipy.stats import pearsonr, binned_statistic, sem, percentileofscore

import pandas as pd
from datetime import datetime
import cPickle as pkl
import os

from matplotlib import rc

import h5py
import lab_repo.misc.lfp_helpers as lfp

# Set rc-params
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 7
plt.rcParams['boxplot.boxprops.linewidth'] = 1

rc('font',**{'family':'sans-serif','sans-serif':['helvetica', 'sans-serif']})
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def initializeExperimentGroup():

    pc_kwargs = {'imaging_label': 'mergedmerged',
                 'nPositionBins': 100,
                 'channel': 'Ch2',
                 'demixed': False,
                 'pf_subset': None,
                 'signal': 'spikes'}

    path = '/home/sebi/grps/dend_grp.json'
    eset = dbExperimentSet(project_name='sebi')
    grp = pcExperimentGroup.from_json(path, eset, **pc_kwargs)

    return grp

def group_order(x):
    
    if x['order'] < 7:
        return x['order']
    else:
        return 7
    
def clean(ax, full=False, offset=0):
    if full:
        sns.despine(ax=ax, left=True, bottom=True, offset=offset)
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        sns.despine(ax=ax, offset=offset)

In [ ]:
grp = initializeExperimentGroup()
grp = pcExperimentGroup([x for x in grp if x.condition==1], **grp.args)

## PSTHs

In [ ]:
# Precomputed in dendrites_repo/analysis_scripts/ripple_transient_psth.py
with open('/home/sebi/psth_soma_trans_1.pkl', 'rb') as fp:
    psth = pkl.load(fp)

UP_SAMPLE = 1.

im_period = grp[0].frame_period()
super_period = im_period / UP_SAMPLE

PRE = 1.0
POST = 3.0

pre = int(PRE / super_period) + 1
post = int(POST / super_period) + 1

fig = plt.figure()
ax = fig.add_subplot(111)

soma_psth_mean = np.nanmean(psth, axis=0)

soma_filtered_mean = gaussian_filter1d(soma_psth_mean[1:-1], 2)
soma_psth_sem = sem(psth, axis=0, nan_policy='omit')[1:-1]
soma_psth_sem = gaussian_filter1d(soma_psth_sem, 2)

T = np.arange(-1*(pre - 1), (post - 1) + 1) * super_period
T = T

colors = ['k', '0.5']

ax.plot(T, soma_filtered_mean, color=colors[0])
ax.fill_between(T, y1=soma_filtered_mean - soma_psth_sem, y2=soma_filtered_mean + soma_psth_sem, alpha=0.5, color=colors[1])

ax.axvline(0, color='r', ls='--')

formatPlot(ax)
ratio = 1.25
ax.set_aspect(ratio / ax.get_data_ratio())
# fig.savefig('/home/sebi/plots/dends/soma_psth_trans.svg')

In [ ]:
# Precomputed in dendrites_repo/analysis_scripts/ripple_transient_psth.py

with open('/home/sebi/psth_dend_trans_1.pkl', 'rb') as fp:
    psth = pkl.load(fp)
    
UP_SAMPLE = 1.

im_period = grp[0].frame_period()
super_period = im_period / UP_SAMPLE

PRE = 1.0
POST = 3.0

pre = int(PRE / super_period) + 1
post = int(POST / super_period) + 1

fig = plt.figure()
ax = fig.add_subplot(111)

psth_mean = np.nanmean(psth, axis=0)

filtered_mean = gaussian_filter1d(psth_mean[1:-1], 2)
psth_sem = sem(psth, axis=0, nan_policy='omit')[1:-1]
psth_sem = gaussian_filter1d(psth_sem, 2)


T = np.arange(-1*(pre - 1), (post - 1) + 1) * super_period
T = T

colors = ['k', '0.5']

ax.plot(T, filtered_mean, color=colors[0])
ax.fill_between(T, y1=filtered_mean - psth_sem, y2=filtered_mean + psth_sem, alpha=0.5, color=colors[1])

ax.axvline(0, color='r', ls='--')

formatPlot(ax)
ratio = 1.25
ax.set_aspect(ratio / ax.get_data_ratio())
# fig.savefig('/home/sebi/plots/dends/dend_psth_trans.svg')

In [ ]:
# Precomputed in dendrites_repo/analysis_scripts/ripple_psth_superresolution_with_shuffle.py

with open('/home/sebi/psth_soma_all_4.pkl', 'rb') as fp:
    psth = pkl.load(fp)
    
UP_SAMPLE = 4.

im_period = grp[0].frame_period()
super_period = im_period / UP_SAMPLE

PRE = 1.5
POST = 2

pre = int(PRE / super_period) + 1
post = int(POST / super_period) + 1

fig = plt.figure()
ax = fig.add_subplot(111)

psth_mean = np.nanmean(psth, axis=0)[:-1]

soma_filtered_mean = gaussian_filter1d(psth_mean, 2)[14:-28]
soma_psth_sem = sem(psth, axis=0, nan_policy='omit')[:-1]
soma_psth_sem = gaussian_filter1d(soma_psth_sem, 2)[14:-28]

soma_T = np.arange(-1*(pre - 1), (post - 1) + 1) * super_period
soma_T = soma_T[13:-28]

colors = ['k', '0.5']

ax.plot(soma_T, soma_filtered_mean, color=colors[0])
ax.fill_between(soma_T, y1=soma_filtered_mean - soma_psth_sem, y2=soma_filtered_mean + soma_psth_sem, alpha=0.5, color=colors[1])

ax.axvline(0, color='r', ls='--')

formatPlot(ax)
ratio = 1.25
ax.set_aspect(ratio / ax.get_data_ratio())

ylims = ax.get_ylim()
# fig.savefig('/home/sebi/plots/dends/soma_psth.svg')

In [ ]:
# Precomputed in dendrites_repo/analysis_scripts/ripple_psth_superresolution_with_shuffle.py

with open('/home/sebi/psth_dend_all_4.pkl', 'rb') as fp:
    psth = pkl.load(fp)
    
UP_SAMPLE = 4.

im_period = grp[0].frame_period()
super_period = im_period / UP_SAMPLE

PRE = 1.5
POST = 2

pre = int(PRE / super_period) + 1
post = int(POST / super_period) + 1

fig = plt.figure()
ax = fig.add_subplot(111)

psth_mean = np.nanmean(psth, axis=0)[:-1]

soma_filtered_mean = gaussian_filter1d(psth_mean, 2)[14:-28]
soma_psth_sem = sem(psth, axis=0, nan_policy='omit')[:-1]
soma_psth_sem = gaussian_filter1d(soma_psth_sem, 2)[14:-28]

soma_T = np.arange(-1*(pre - 1), (post - 1) + 1) * super_period
soma_T = soma_T[13:-28]

colors = ['k', '0.5']

ax.plot(soma_T, soma_filtered_mean, color=colors[0])
ax.fill_between(soma_T, y1=soma_filtered_mean - soma_psth_sem, y2=soma_filtered_mean + soma_psth_sem, alpha=0.5, color=colors[1])

ax.axvline(0, color='r', ls='--')

formatPlot(ax)
ratio = 1.25
ax.set_aspect(ratio / ax.get_data_ratio())

ylims = ax.get_ylim()
# fig.savefig('/home/sebi/plots/dends/dend_psth.svg')

## PSTH Diff

In [ ]:
plotdir= '/home/sebi/plots/dends/fig1'

In [ ]:
with open('/home/sebi/psth_soma_all_4_by_cell.pkl', 'rb') as fp:
    soma_psth = pkl.load(fp)

UP_SAMPLE = 4.

im_period = grp[0].frame_period()
super_period = im_period / UP_SAMPLE

PRE = 1.5
POST = 2

pre = int(PRE / super_period) + 1
post = int(POST / super_period) + 1
sec = int(0.5 / super_period)

soma_vals = []
for p in soma_psth:
    soma_vals.append(np.nanmean(p[pre:pre+sec]) - np.nanmean(p[pre-sec:pre]))

with open('/home/sebi/psth_soma_all_4_by_cell_shuffles.pkl', 'rb') as fp:
    shuffles = pkl.load(fp)
    
shuffle_means = [np.nanmedian(x) for x in shuffles]

In [ ]:
fig = plt.figure(figsize=(1, 0.5), dpi=200)
ax = fig.add_subplot(111)

ax.hist(shuffle_means, bins=50, density=False)

ax.axvline(np.nanmedian(soma_vals), color='r', lw=1)
clean(ax, offset=1)

ax.set_ylabel('N Shuffles')
ax.set_xlabel('Median Post-Pre')

fig.savefig(os.path.join(plotdir, 'soma_shuffles.svg'))

In [ ]:
percentileofscore(shuffle_means, np.nanmedian(soma_vals))

In [ ]:
with open('/home/sebi/psth_dend_all_4_by_cell.pkl', 'rb') as fp:
    dend_psth = pkl.load(fp)

UP_SAMPLE = 4.

im_period = grp[0].frame_period()
super_period = im_period / UP_SAMPLE

PRE = 1.5
POST = 2

pre = int(PRE / super_period) + 1
post = int(POST / super_period) + 1
sec = int(0.5 / super_period)

dend_vals = []
for p in dend_psth:
    dend_vals.append(np.nanmean(p[pre:pre+sec]) - np.nanmean(p[pre-sec:pre]))

with open('/home/sebi/psth_dend_all_4_by_cell_shuffles', 'rb') as fp:
    dend_shuffles = pkl.load(fp)
    
dend_shuffle_means = [np.nanmedian(x) for x in dend_shuffles]

In [ ]:
fig = plt.figure(figsize=(1, 0.5), dpi=200)
ax = fig.add_subplot(111)

ax.hist(dend_shuffle_means, bins=50, density=False, color='0.7')

ax.axvline(np.nanmedian(dend_vals), color='r', lw=1)
clean(ax, offset=1)
ax.set_ylabel('N Shuffles')
ax.set_xlabel('Median Post-Pre')

print percentileofscore(dend_shuffle_means, np.nanmedian(dend_vals))

fig.savefig(os.path.join(plotdir, 'dend_shuffles.svg'))

## Ripple Spectrogram

In [ ]:
fs = 1250
pre = 0.5
post = 0.5

pre = int(pre * fs / 4)
post = int(post * fs / 4)

specs = []

for expt in grp:
    rtimes = np.array(expt.ripple_times()) / 4
    rframes = expt.ripple_frames()
    
    running = expt.runningIntervals(returnBoolList=True)[0]
        
    if rtimes.shape[0] == 0:
        continue
    
    try:
        d, f = lfp.get_spectro(expt)
    except AttributeError:
        continue
                    
    for rtime, rframe in zip(rtimes, rframes):
        if running[rframe]:
            continue
        specs.append(d[:, rtime-pre:rtime+post])

In [ ]:
fig = plt.figure(figsize=(1.25, 1.25), dpi=200)
ax = fig.add_subplot(111)

plotspecs = [x for x in specs if x.shape == (70, pre+post)]
print len(plotspecs)

mean_spec = np.nanmean(plotspecs, axis=0)

sns.heatmap(mean_spec, vmax=20, vmin=0, cbar=True, rasterized=True,
            cbar_kws={'ticks': [0, 20], 'label': 'Normalized PSD'})

clean(ax)


include_ticks = range(9, 70, 10)[::-1]
tick_labels = [str(int(f[x][0])) for x in include_ticks]

for axis in ['left', 'right', 'bottom', 'top']:
    ax.spines[axis].set_linewidth(1)
    ax.spines[axis].set_visible(True)
    ax.tick_params(labelsize=6, length=2, width=1)

ax.set_yticks(include_ticks)
ax.set_yticklabels(tick_labels, rotation=0)
ax.invert_yaxis()
ax.set_ylim([5, 60])
ax.set_xticks([0, 156, 312])
ax.set_xticklabels([-0.5, 0, 0.5], rotation=0)

ax.set_ylabel('Frequency (Hz)')
ax.set_xlabel('Time from SWR Onset (s)')

fig.savefig(os.path.join(plotdir, 'spectrogram.svg'))